In [1]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

### Train Batch Pre-Processing

In [2]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"

with open('Interviewdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    questions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(questions))
with open('Interviewdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    answers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(answers))
with open('Interviewdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    labels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(labels)) 

5294
5294
5294


In [3]:
completedata = questions.copy()
for i in answers:
    completedata.append(i)

In [4]:
len(completedata)

10588

In [5]:
import pickle
with open('questions.txt', 'wb') as f:
    pickle.dump(completedata, f)

In [6]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [7]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [8]:
wordlist.append('UNKNOWN_TOKEN')

In [9]:
len(wordlist)

2627

In [10]:
embeddings = np.vstack([embeddings, np.zeros(200)])

In [11]:
len(embeddings)

2627

In [12]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [13]:
len(word_to_index)

2627

In [14]:
for i, sent in enumerate(questions):
    questions[i] = [w if w in word_to_index else unknown_token for w in sent]

In [15]:
for i, sent in enumerate(answers):
    answers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [16]:
import numpy as np
X_question = np.asarray([[word_to_index[w] for w in sent] for sent in questions])
y_answer = np.asarray([[word_to_index[w] for w in sent] for sent in answers])

In [23]:
X_question[:10]

array([ [6, 7, 149, 0, 63, 2, 41, 314, 73, 74, 581, 73, 74, 217, 149, 846, 603, 528, 865, 10],
       [6, 7, 149, 0, 63, 2, 19, 54, 1077, 3, 475, 173, 849, 5, 1109, 3, 224, 5, 1163, 64, 23, 64, 101, 493, 5, 0, 54, 1, 6, 32, 223, 493, 5, 54, 243, 120, 15, 619, 1],
       [12, 2, 410, 70], [12, 2, 251, 763],
       [591, 264, 888, 73, 74, 6, 35, 47, 46, 28, 175, 465, 96, 591, 11, 8, 294, 1148, 223, 12, 27, 15, 401, 10],
       [33, 848, 0, 63, 2, 111, 41, 314, 73, 74, 363, 11, 70, 794, 8, 73, 74, 363, 11, 447, 805, 8, 73, 74, 12, 27, 15, 401, 4, 41, 63, 10],
       [32, 17, 2, 0, 1084, 641, 17, 5, 70, 17],
       [1, 46, 0, 63, 2, 111, 41, 202, 1151, 202, 11, 8, 30, 202, 1178, 30, 202, 1185, 30, 991, 1, 1173, 30, 12, 27, 401, 4, 744, 10],
       [40, 230, 13, 492, 35, 20, 2, 833, 19, 117, 56, 10],
       [12, 2, 306, 22, 10]], dtype=object)

In [24]:
x_example, y_example = X_question[0], y_answer[0]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
in a program the code is this try { } finally { } whether program compiles & runs successfully ?
[6, 7, 149, 0, 63, 2, 41, 314, 73, 74, 581, 73, 74, 217, 149, 846, 603, 528, 865, 10]

y:
yes , UNKNOWN_TOKEN can be possible without catch block
[165, 3, 2626, 13, 15, 182, 294, 363, 223]


In [25]:
import numpy as np

x = X_question
y = y_answer
max_length =  45
x_padded = []
y_padded = []


for row in x:
    if len(row) <= max_length:
        x_padded.append(row + [2626] * (max_length - len(row)))
    else :
        x_padded.append(row[:45])

for row in y:
    if len(row) <= max_length:
        y_padded.append(row + [2626] * (max_length - len(row))) 
    else: 
        y_padded.append(row[:45])


### Test Batch Pre-Processing

In [48]:
with open('Interviewtestdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testquestions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testquestions))

with open('Interviewtestdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testanswers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testanswers)) 

with open('Interviewtestdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testlabels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(testlabels))

4
4
4


In [49]:
for i, sent in enumerate(testquestions):
    testquestions[i] = [w if w in word_to_index else unknown_token for w in sent]
    
for i, sent in enumerate(testanswers):
    testanswers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [50]:
X_testquestion = np.asarray([[word_to_index[w] for w in sent] for sent in testquestions])
y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])

In [51]:
max_length = 45
x_testpadded = []
y_testpadded = []
p = X_testquestion
q = y_testanswer

for row in p:
    if len(row) <= max_length:
        x_testpadded.append(row + [2626] * (max_length - len(row))) 
    else :
        x_testpadded.append(row[:45])

for row in q:
    if len(row) <= max_length:
        y_testpadded.append(row + [2626] * (max_length - len(row)))
    else :
        y_testpadded.append(row[:45])

In [52]:
len(x_testpadded), len(y_testpadded)

(4, 4)

In [43]:
tf.reset_default_graph()

from tensorflow.contrib import rnn 
hm_epochs = 50
batch_size = 4
state_size = 200
vocab_size = 2627
num_steps = 45

# Placeholders
context = tf.placeholder(tf.int32, [batch_size, num_steps], name = "context") # [batch_size, num_steps]
utterance = tf.placeholder(tf.int32, [batch_size,num_steps], name = "utterance")
targets = tf.placeholder(tf.int32, [batch_size], name = "targets")
target = tf.placeholder(tf.int32, [batch_size], name = "target")

def next_batch(step):
    p,q = x_padded[batch_size*step:batch_size*(step+1)], y_padded[batch_size*step:batch_size*(step+1)]
    t = labels[batch_size*step:batch_size*(step+1)]
    return p,q,t

def test_batch(step):
    a,b = x_testpadded[batch_size*step:batch_size*(step+1)], y_testpadded[batch_size*step:batch_size*(step+1)]
    e = testlabels[batch_size*step:batch_size*(step+1)]
    return a,b,e
        
def dual_encoder_model(context,utterance,targets):
    
    #Embedding the question and answer
    context_embedded = tf.nn.embedding_lookup(embeddings, context, name="embed_context")
    utterance_embedded = tf.nn.embedding_lookup(embeddings, utterance, name="embed_utterance")
    print(" This is the embeddings for both Q&A:", context_embedded, utterance_embedded)
    
    with tf.variable_scope("rnn") as vs:
        cell = tf.contrib.rnn.BasicLSTMCell(state_size)
        cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5) 
        #cell = tf.contrib.rnn.MultiRNNCell(cells= [cell]*3, state_is_tuple= True)
    
    #feed inputs to dynamic rnn for training    
    rnn_inputs = tf.concat([context_embedded, utterance_embedded],0)
    rnn_inputs = tf.transpose(rnn_inputs, [1,0,2])
    rnn_inputs = tf.reshape(rnn_inputs, [-1, state_size])
    rnn_inputs = tf.split(rnn_inputs, num_steps, 0)
    
    rnn_outputs, rnn_states = rnn.static_rnn(cell, rnn_inputs, dtype=tf.float64)
    
    encoding_context, encoding_utterance = tf.split(rnn_states.h,2,0)
    
    print("\nThis is the encoding context:", encoding_context)
    print("\nThis is the encoding utterance:", encoding_utterance)
    
    with tf.variable_scope("prediction") as vs:
        M = tf.get_variable("M",
          shape=[state_size, state_size],
          initializer=tf.truncated_normal_initializer())

    # "Predict" a  response: c * M
    generated_response = tf.matmul(encoding_context, tf.cast(M, dtype = tf.float64))
    print("\nThis is the generated response:", generated_response)
    generated_response = tf.expand_dims(generated_response, 2)
    print("\nThis is the generated response:", generated_response)
    encoding_utterance = tf.expand_dims(encoding_utterance, 2)
    print("\nThis is the encoding utterance:", encoding_utterance)
    # Dot product between generated response and actual response
    # (c * M) * r
    logit = tf.matmul(generated_response, encoding_utterance, True)
    print("\nThis is the logits before squeeze:", logit)
    logits = tf.squeeze(logit, [2])
    print("\nThis is the logits after squeeze:", logits)

    # Apply sigmoid to convert logits to probabilities
    probs = tf.sigmoid(logits, name = "Sigmoid")   
    print("\nThis is the probability after sigmoid:", probs)
    return tf.reshape(logits, [-1]), tf.reshape(probs, [-1])

def train_encoder(x):
    prediction, probs = dual_encoder_model(context,utterance,targets)
    # Calculate the binary cross-entropy loss
    print("\nThis is the prediction size:", prediction)
    print("\nThis is the target size:", targets)
    losses = tf.nn.sigmoid_cross_entropy_with_logits(logits = prediction, labels = tf.cast(targets, dtype = tf.float64))

    # Mean loss across the batch of examples
    cost = tf.reduce_mean(losses, name="cost")
    optimizer = tf.train.AdamOptimizer().minimize(cost)   
    saver = tf.train.Saver()
        
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(hm_epochs):
            epoch_loss = 0 
            for step in range(int(len(x_padded)/batch_size)):
                ques, ans,labl = next_batch(step)
                _, c, logts, prob = sess.run([optimizer, cost, prediction, probs], feed_dict={context: ques,
                                                              utterance: ans,  
                                                              targets: labl})
                epoch_loss += c
                #print('step:', step, 'logits:', logts, 'probability:', prob)
            print('***************************** Epoch Completed *************************************')
            print('Epoch:', epoch, 'completed out of:',hm_epochs,'loss:',epoch_loss, 'probability:', prob)
        save_path = saver.save(sess, "./Interviewmodel")
        print("Model saved in file: %s" % save_path)  
        qe, an,lbl = test_batch(0)
        probas = sess.run(probs, feed_dict={context: qe, utterance: an, target: lbl})
        print("Probs:", probas)

In [44]:
train_encoder(x)

 This is the embeddings for both Q&A: Tensor("embed_context:0", shape=(4, 45, 200), dtype=float64) Tensor("embed_utterance:0", shape=(4, 45, 200), dtype=float64)

This is the encoding context: Tensor("split_1:0", shape=(4, 200), dtype=float64)

This is the encoding utterance: Tensor("split_1:1", shape=(4, 200), dtype=float64)

This is the generated response: Tensor("MatMul:0", shape=(4, 200), dtype=float64)

This is the generated response: Tensor("ExpandDims:0", shape=(4, 200, 1), dtype=float64)

This is the encoding utterance: Tensor("ExpandDims_1:0", shape=(4, 200, 1), dtype=float64)

This is the logits before squeeze: Tensor("MatMul_1:0", shape=(4, 1, 1), dtype=float64)

This is the logits after squeeze: Tensor("Squeeze:0", shape=(4, 1), dtype=float64)

This is the probability after sigmoid: Tensor("Sigmoid:0", shape=(4, 1), dtype=float64)

This is the prediction size: Tensor("Reshape_1:0", shape=(4,), dtype=float64)

This is the target size: Tensor("targets:0", shape=(4,), dtype=in

In [53]:
with open('Interviewtestdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testanswers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testanswers)) 
    
for i, sent in enumerate(testanswers):
    testanswers[i] = [w if w in word_to_index else unknown_token for w in sent]
    
y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])

max_length = 45
y_testpadded = []
for row in y_testanswer:
    if len(row) <= max_length:
        y_testpadded.append(row + [2626] * (max_length - len(row)))
    else :
        y_testpadded.append(row[:45])

4


In [54]:
for i in y_testpadded:
    print("x:\n%s\n\n%s" % (" ".join([wordlist[x] for x in i]),i))

x:
no UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN

[95, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626, 2626]
x:
i UNKNOWN_TOKEN not sure what are you UNKNOWN_TOKEN , a block ? UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN

In [55]:
#def test_batch(step):
    #a,b = x_testpadded[batch_size*step:batch_size*(step+1)], y_testpadded[batch_size*step:batch_size*(step+1)]
    #e = testlabels[batch_size*step:batch_size*(step+1)]
    #return a,b,e

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('model.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
sess.run(tf.global_variables_initializer())
graph = tf.get_default_graph()

context = graph.get_tensor_by_name("context:0") 
utterance = graph.get_tensor_by_name("utterance:0")
target = graph.get_tensor_by_name("target:0")

op_to_restore = graph.get_tensor_by_name("Sigmoid:0")
#qe,an,lbl = test_batch(0)
feed_dict={context: x_testpadded, utterance: y_testpadded, target: testlabels}
print(sess.run(op_to_restore,feed_dict))

[[ 0.5000009 ]
 [ 0.50017393]
 [ 0.4991018 ]
 [ 0.50001364]]
